# Merapikan data Digilib

In [33]:
import os
import re
import random
import itertools as it
from pickle import load

from unicodedata import normalize as utf8_normalize

In [2]:
total = []

for root, dirs, files in os.walk('collected-picklep-digilib/collected/'):
    for file in files:
        fname = os.sep.join([root, file])
        
        try:
            with open(fname, 'rb') as f: total.extend(load(f))
        except EOFError: pass

In [3]:
kontributor = [row['Kontributor / Dosen Pembimbing'] for row in total]
del total

In [4]:
# simpan data original
original = kontributor.copy()

In [5]:
len(kontributor)

57406

## Remove "Stopwords"

Pada bagian ini, kita akan melihat dan menghilangkan beberapa kata (kumpulan karakter yang diapit oleh simbol spasi) yang sering muncul dalam data.

In [6]:
all_word = ' '.join(kontributor)
all_word = re.sub('(\:|\,|;)', ' ~ ', all_word).split()

In [7]:
len(all_word)

552265

Coba hitung frekuensi kemunculan dari setiap kata dalam data

In [8]:
word_freq = {}

while all_word:
    word = all_word.pop()
    if word not in word_freq: word_freq[word] = 0
    word_freq[word] += 1

In [9]:
tmp = sorted([(freq,word) for word,freq in word_freq.items()], reverse=True)[:50]

# print([word for freq,word in tmp])

stop_word_fp = {'-', 'Dr.', 'Pembimbing', 'Ir.', 'Ph.D.', 'S.T.', 'M.T.', 'Prof.', 'M.Sc.', 
                'M.Si.', 'S.Si.', 'M.Eng.', 'Ph.D', 'MT.', 'dan', 'Scan', 'Dr.Eng.', 'ST.',
                '1', 'Scanner', '2', '#CONTRIBUTOR#', 'Dr.Ir.', 'Drs.', 'Dr.-Ing.', 'Eng.',
                'M.Sn.', 'Dr.rer.nat.', 'Apt.', 'M.Sc',}

print([word for freq,word in tmp if word not in stop_word_fp])

['~', 'M.', 'I', 'MS', 'Bambang', 'P', 'S.', 'Agus', 'MBA', 'MT', 'Muhammad', 'Budi', 'A.', 'DEA', 'ST', 'Hidayat', 'Achmad', 'R.', 'Tri', 'Gunawan']


Beberapa kata yang sering muncul saya simpan di `stop_word_fp`. Namun tidak semuanya saya masukkan. Kata yang ambigu seperti `MS` dapat berarti singkatan nama, seperti `Muhammad Saiful`, juga dapat berupa singkatan gelar. Karenanya, saya hanya memasukkan kata yang kemungkinan besar, hampir pasti, bukan merupakan singkatan nama. Tentu, jika `MT.` sebenarnya singkatan nama dari seseorang... maaf.

In [10]:
# kita tidak membutuhkan ini lagi
del word_freq

Dengan melihat 100 sampel acak beberapa kali, saya melihat beberapa pola yang dapat sekalian bersihkan. Contohnya adalah teks `Dosen Pembimbibing I` dan `ko-supervisor 2:`. Daftar lengkapnya dapat dilihat berikut

In [11]:
# hilangkan stop word yang tadi kita kumpulkan
remove1 = r'\W(%s)\W' % r'|'.join(stop_word_fp)
# hilangkan simbol ini yang setidaknya muncul dua kali secara berurutan
remove2 = r'[\d\t\n\(\)\-\,;: ]{2,}'
# ... muncul sekali
remove3 = r'[&\,]'
# hilangkan simbol non karakter di awal dan di akhir data
remove4 = r'^\W+'
remove5 = r'\W+$'

# pola yang saya lihat dan perlu dihapus
noname1 = '(dosen)?( ){,3}(pe.{4,8}ng|advisor|akademik)( )*(utama|tesis|pertama|kedua|proyek akhir|tugas akhir|I{,3}|\d)( )*:?'
noname2 = '(|ko-|co-)(anggota|supervisor|promotor|penulis|pembina|author(s)?)[ I\d]*?:?'
noname3 = 'Koordinator (Kelompok|Tugas Akhir|TA Desain Produk)'
noname4 = 'Ketua( Program Studi)?'
noname5 = 'scan(ner)?[ :]*.*'
noname6 = '(Ena Sukmana|editor| oleh |unggah pertama pada)'

# hapus penggunaan gelar yang sangat umum muncul
title1 = 'Dr(a|s)?.*?(nat|pol|tech(n)?)( |\.)'
title2 = 'Dr(a|s)?.{,3}(ing|eng|ir)( |\.)'
title3 = 'Dr(a|s)?\.'
title4 = 'Ph( |\.|\. )D( |\.)'
title5 = 'PhD( |\.)'
title6 = '(Prof|Ir)\.'
title7 = 'M( |\.|\. )B( |\.|\. )A( |\.|\. )'

# hilangkan spasi ganda
cleanup1 = '\s+'
# menyederhanakan apa yang telah dihapus
cleanup2 = '~( ~)+'

Lalu saya membuat fungsi besar untuk merapikan data

In [12]:
def strip(tmp):
    # inisiasi, seharusnya bisa lebih efektif
    tmp = ' ' + tmp + ' '
    # rapikan penggunaan karakter ke bentuk yang lebih sederhana
    tmp = utf8_normalize('NFKD', tmp)

    # untuk setiap pola regex yang ditemukan, ganti dengan ' ~ ' 
    for pattern in [remove1, remove2, remove3, remove4, remove5,
            noname1, noname2, noname3, noname4, noname5, noname6,
            title1, title2, title3, title4, title5, title6, title7]:
        tmp = re.sub(pattern, ' ~ ', tmp, flags=re.IGNORECASE)
    
    # cleanup
    tmp = re.sub(cleanup1, ' ', tmp)
    tmp = re.sub(cleanup2, '~', tmp)

    # sekarang, ' ~ ' memisahkan dua nama lengkap yang berbeda.
    # kumpulkan daftar nama lengkap menjadi sebuah list
    get = []
    for name in tmp.split(' ~ '):
        # tentu, selama nama lengkap bukan teks kosong, atau email@somewhere.com
        if name and ('@' not in name):
            get.append(name)
    return get

Mari kita cek beberapa kasus

In [13]:
tests = ['Dumaria R. Tampubolon',
         'Pembimbing: Dr. Irawati',
         'H.S. Sabet, A.N. El-Sayed, M.A. Zayed',
         'Nugraha, Dr.Eng.;Dr. Damar Rastri Adhika, S.T., M.Sc.;',
         'Pembimbing: Dr. Ir. Budi Hartono Setiamarga dan Umen Rumendi, S.T., Weld. Mast.']

for test in tests:
    print(strip(test))

['Dumaria R. Tampubolon']
['Irawati']
['H.S. Sabet', 'A.N. El-Sayed', 'M.A. Zayed']
['Nugraha', 'Damar Rastri Adhika']
['Budi Hartono Setiamarga', 'Umen Rumendi', 'Weld. Mast']


Terlihat oke. Sayangnya gelar `Weld. Mast` tercatat sebagai nama lengkap. Perbaikan ini bisa dilakukan pada tahap selanjutnya. Saat ini, mari kita bersihkan data terlebih dahulu

In [14]:
for i in range(len(kontributor)):
    kontributor[i] = strip(kontributor[i])

## Menghapus Gelar dari Nama Lengkap

Untuk daftar gelar akademik, saya mengambil data dari [Halaman Wikipedia berikut](https://id.wikipedia.org/wiki/Gelar_akademik). Lebih lanjut, dalam membersihkan gelar, kita akan berurusan dengan nama seperti `M.A. Sayed`, dan memikirkan untuk membuat daftar gelar yang perlu diletakkan sebelum, dan daftar yang perlu diletakkan sesudah nama lengkap. Untuk alasan kemudahan, saya hanya memisahkan daftar gelar secara sederhana

In [15]:
# gelar akademik yang terletak di awal
acad_title_start = {'Dr.', 'Dr. Psi.', 'Dr. iur.', 'Dr. med.', 'Dr. phil.', 'Dr. rer. nat.',
    'Dr. rer. oec.', 'Dr. rer. pol.', 'Dr.-Ing.', 'Dra.', 'Drs.', 'Ir.', 'Mr.', 'Prof.'}

# ... yang terletak di akhir
acad_title_end = {'A.Ma.', 'A.Md.', 'A.P.', 'B.A.', 'B.Ba.', 'B.Biomed.', 'B.Comp.Sc.', 'B.Eng.', 'B.Sc.',
    'D.Th./Th.D.', 'Ed.D.', 'H./Dr.', 'Hk.',
    'L.L.B.', 'L.L.M.', 'M.A.', 'M.A.B.', 'M.A.P.', 'M.A.R.S.', 'M.Arl.', 'M.B.A.', 'M.Com.', 'M.Cs.',
    'M.E.', 'M.Eng.', 'M.Eng.Sc.', 'M.I.Kom.', 'M.ISS', 'M.Int.Rel.', 'M.Kom.', 'M.M.A.S.', 'M.M.D.S.',
    'M.Mar.', 'M.Mar.E.', 'M.Mgt.', 'M.Pharm.', 'M.Phil.', 'M.Psi., Psikolog/M.Psi., Psi.', 'M.Psi.T.',
    'M.S./M.Sc.', 'M.S.S.', 'M.Sc.', 'M.Si.', 'M.T.', 'M.Th.', 'MBBS', 'Ph.D.', 'S.A P.', 'S.A.B.',
    'S.A.P.', 'S.A.R.S.', 'S.Adm.', 'S.Ag.', 'S.Agr.', 'S.Ant.', 'S.Arl.', 'S.Ars.', 'S.Ds.', 'S.E.',
    'S.E.I.', 'S.Farm.', 'S.Fil.', 'S.Gz./S.Gizi', 'S.H.', 'S.H.I.', 'S.Han.', 'S.Hub.Int.', 'S.Hum.',
    'S.Hut.', 'S.I.K.', 'S.I.Kom.', 'S.I.P.', 'S.I.P./S.I.Pol.', 'S.I.P./S.Ptk.', 'S.In.', 'S.K.G.',
    'S.K.H.', 'S.K.M.', 'S.K.P.M.', 'S.Keb.', 'S.Ked.', 'S.Kel.', 'S.Kep.', 'S.Kom.', 'S.M./S.Mn.',
    'S.M.B.', 'S.Mat.', 'S.Or.', 'S.P.', 'S.Par.', 'S.Pd.', 'S.Pd.Bio', 'S.Pd.Fis', 'S.Pd.I.', 'S.Pd.Jas',
    'S.Pd.Kor', 'S.Pd.SD.', 'S.Pi.', 'S.Psi.', 'S.Pt.', 'S.S.', 'S.S.T.', 'S.S.T.Han./S.T.Han./S.Tr.Han.',
    'S.SI.', 'S.STP.', 'S.Si.', 'S.Si. (Teol.)', 'S.Sn.', 'S.Sos.', 'S.Sy.', 'S.T.', 'S.T.P.', 'S.TI.',
    'S.Th.', 'S.Th.I.', 'S.Tr.K.', 'S.Tr.Keb.', 'S.Tr.Sos.', 'Th.M.'}

Tentu, pada kenyataannya kita melihat banyak sekali variasi dari penulisan nama gelar tersebut. Contohnya, gelar `S.E.` dapat mengalami saltik menjadi `SE.`, `S.E`, dan bahkan `SE`. Saya mengambil langkah konservatif dengan hanya menambah kemungkinan saltik "lupa menambahkan titik di akhir gelar" ke dalam daftar gelar.

In [16]:
acad_title_start.update([x[:-1] for x in acad_title_start if x[-1]=='.'])
acad_title_end.update(  [x[:-1] for x in acad_title_end   if x[-1]=='.'])

In [17]:
def strip(text):
    text = text.split(' ')
    
    # hilangkan gelar di akhir nama
    for i in reversed(range(len(text))):
        if text[i] not in acad_title_end: break
    if text[i] in acad_title_end: i-=1
    
    # hilangkan gelar di awal nama
    for j in range(max(len(text), i)):
        if text[j] not in acad_title_start: break
    if text[j] in acad_title_start: j+=1
        

    # gabungkan
    text = ' '.join(text[j:i+1])
    
    return text

In [18]:
tests = ['Dr M.A Zayed', 'Ir. Emenda Sembiring S.T. M.T. M.Eng.Sc. Ph.D.', 'S.T']

for test in tests:
    print(strip(test))

M.A Zayed
Emenda Sembiring



Sekali lagi, rapikan data yang kita miliki

In [19]:
for i in range(len(kontributor)):
    revised_list = []
    for name in kontributor[i]:
        revised = strip(name)
        # jika revised bukan string kosong
        if revised:
            revised_list.append(revised)
    kontributor[i] = revised_list

### Perkembangan sejauh ini

In [20]:
all_word = []

for names in kontributor:
    all_word.append(' '.join(names))
all_word = ' '.join(all_word).split()

In [21]:
len(all_word)

213104

In [22]:
word_freq = {}

while all_word:
    word = all_word.pop()
    if word not in word_freq: word_freq[word] = 0
    word_freq[word] += 1
del all_word

In [23]:
tmp = sorted([(freq,word) for word,freq in word_freq.items()], reverse=True)

stop_word_sp = {word for freq,word in tmp
                if word in acad_title_start.union(acad_title_end)}

print(sorted(stop_word_sp))

['A.P.', 'B.A.', 'Dr', 'Ir', 'M.A', 'M.A.', 'M.Phil', 'M.Sc', 'M.Si', 'M.T', 'MBBS', 'Prof', 'S.Ds', 'S.H.', 'S.P', 'S.P.', 'S.Psi', 'S.S.', 'S.Sn.', 'S.Sos.']


In [24]:
stop = False
for ori, rev in zip(original, kontributor):
    for name in rev:
        if 'Prof' in name.split(' '):
            print(ori,'\n', rev)
            stop = True
            break
    if stop: break

Pembimbing: Kudrat Soemintapoera. Prof.Ir.,Ph.D., dan Suhardi, Dr. Ing. 
 ['Kudrat Soemintapoera. Prof', 'Suhardi', 'Ing']


In [25]:
del word_freq

## Periksa Nama yang Sering Muncul

In [26]:
name_freq = {}

for revlist in kontributor:
    for name in revlist:
        if name not in name_freq:
            name_freq[name] = 0
        name_freq[name] += 1

In [27]:
tmp = sorted([(freq,word) for word,freq in name_freq.items()], reverse=True)[:150]

# Saya menganggap nama berikut sebagai left-over dari gelar
stop_word_tp = {'MS', 'P', 'MBA', 'MT', 'DEA', 'I', 'II', 'S', 'Ing.', 'MSc', 'Dosen', 'MA', 'M.S.M', 'dan',
                'M.Ds', 'Tim', 'MSCE', 'MM', 'MSP', 'MSME', 'M.M', 'Utama', 'MSc.', 'MSE', 'M.SE', 'M.S', 'MSA',
                'S. Sn.', 'M. T', 'M.Pl', 'IEEE',
                'MS.', 'MSIE', 'M', 'M.P', 'M.A.Sc', 'ena', 'Eng.', 'M.Sn', 'MES', 'M.M.', 'DBA', 'upload',
                'M.S.', 'SE', 'DIC', 'M. Sc', 'MAUD', 'Tugas Akhir', 'SE.DEA', 'M. Sn', 'Thesis'}
stop_word_tp.update([word for word in name_freq.keys() if len(word)<=2])

#print([word for _,word in tmp if word not in stop_word_tp])

percentage = sum(freq for freq,word in tmp if word in stop_word_tp) / sum(name_freq.values())
100 * percentage

13.68493555052642

In [28]:
for i in range(len(kontributor)):
    kontributor[i] = [name for name in kontributor[i] if name not in stop_word_tp]

## Approximate String Matching
https://www.machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/

In [29]:
random.sample(kontributor, 100)

[["Zaki Su'ud"],
 ['Deddy Kurniadi', 'Suprijanto'],
 ['Wenbing Guo', 'Erhu Bai', 'and Daming Yang'],
 ['Ing. Willy Adriansyah'],
 ['Suprijadi'],
 [],
 ['Ivonne Milichristi Radjawane'],
 ['Eddy A. Subroto'],
 ['M. Cahyono', 'Arie Setiadi Moerwanto'],
 ['Sasanti Tarini Darijanto'],
 ['Suprayogi', 'Rully Tri Cahyono'],
 [],
 ['Harmein Rahman', 'Bambang Sugeng S.'],
 ['Pudjo Sukarno', 'Leksono Mucharam'],
 [],
 ['Muhammad Bachri Amran'],
 ['Achmad Haldani Destiarmand'],
 ['Tutuka Ariadji'],
 ['H. Rochim Suratman'],
 ['Kinsenary Tjendrasa'],
 ['Atika Lubis'],
 ['Wahyu Srigutomo'],
 ['Wawan Dhewanto'],
 ['Ridho K. Wattimena'],
 ['Bambang Sunendar', 'Ahmad Nuruddin'],
 ['Irawati', 'Muchtadi Intan Detiena', 'Aslak Bakke Buan'],
 [],
 ['Mardjono Siswosuwarno'],
 ['ChangHee Lee', 'L. Mahadevan', 'Clifford J. Tabin'],
 [],
 ['Andojo Wurjanto'],
 ['Leksananto'],
 [],
 ['Bermawi P. Iskandar'],
 ['Ahmad Nuruddin'],
 ['Dumaria Rulina Tampubolon'],
 ['Ignatius Sonny Winardhie'],
 ['Aciek Ida Wuryandar

## NetworkX

In [31]:
import networkx as nx

In [46]:
G = nx.Graph()

In [47]:
for revlist in kontributor:
    if not revlist: continue
    if len(revlist)==1:
        tmp = revlist[0]
        if tmp in G.nodes: G[tmp]['size']+=1
        else: G.add_node(tmp, size=1)
        continue
    
    for x,y in it.combinations(revlist, r=2):
        print(x,y)
    break

Edi Leksono Nugraha
Edi Leksono M. Eng
Nugraha M. Eng


In [48]:
?G.add_node

Signature: G.add_node(node_for_adding, **attr)
Docstring:
Add a single node `node_for_adding` and update node attributes.

Parameters
----------
node_for_adding : node
    A node can be any hashable Python object except None.
attr : keyword arguments, optional
    Set or change node attributes using key=value.

See Also
--------
add_nodes_from

Examples
--------
>>> G = nx.Graph()  # or DiGraph, MultiGraph, MultiDiGraph, etc
>>> G.add_node(1)
>>> G.add_node("Hello")
>>> K3 = nx.Graph([(0, 1), (1, 2), (2, 0)])
>>> G.add_node(K3)
>>> G.number_of_nodes()
3

Use keywords set/change node attributes:

>>> G.add_node(1, size=10)
>>> G.add_node(3, weight=0.4, UTM=("13S", 382871, 3972649))

Notes
-----
A hashable object is one that can be used as a key in a Python
dictionary. This includes strings, numbers, tuples of strings
and numbers, etc.

On many platforms hashable items also include mutables such as
NetworkX Graphs, though one should be careful that the hash
doesn't change on mutables.
Fi